In [1]:
import pandas as pd
import re

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
#Read File Meta Reviews and Geometry
dataframe_MR_RG=pd.read_csv("MR_RG.csv")
dataframe_MR_RG.head(2)

,gmap_id,placekey,location_name,street_address,city,region,postal_code,latitude,longitude,Lead,...,Month,spend_by_transaction_intermediary,online_spend,online_transactions,Week Start,Sum_spent_by_week,Average_spent_per_transaction_by_week,Date,Average Rating,Count of Reviews
0,0x8636d33deda334eb:0x8539697e533cad66,zzw-222@8f2-d5r-4y9,Accucare Medical,9011 Linwood Ave,Shreveport,LA,71106,32.419457,-93.762363,women-led,...,"['2019-02-01', ' 2019-04-01', ' 2019-06-01', '...",{'No intermediary'},0,0,"['2019-01-28', '2019-02-04', '2019-02-11', '20...","[104.3, 34.35, 158.12, 0.0, 70.83, 0.0, 0.0, 0...","[34.77, 4.91, 22.59, 0.0, 17.71, 0.0, 0.0, 0.0...","['2015-11-09', '2016-04-25', '2016-05-16', '20...","[5.0, 1.0, 5.0, 2.0, 5.0, 5.0, 5.0, 1.0, 5.0, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,0x88f5a28359cb48cd:0xacc5f8fde9522f87,zzw-223@8g6-ycf-2ff,Gwinnett Community Bank,2775 Buford Hwy,Duluth,GA,30096,34.007883,-84.133413,No Lead,...,"['2019-03-01', ' 2019-04-01', ' 2019-11-01']",{'No intermediary'},0,0,"['2019-02-25', '2019-03-04', '2019-03-11', '20...","[0.0, 35.0, 25.0, 0.0, 0.0, 325.99, 109.36, 0....","[0.0, 5.0, 3.57, 0.0, 0.0, 46.57, 15.62, 0.0, ...","['2019-07-22', '2020-09-14']","[5.0, 1.0]","[2, 2]"


In [4]:
#Drop Duplicate Rows and keep first row
dataframe_MR_RG=dataframe_MR_RG.drop_duplicates(keep="first")

In [5]:
#Drop Duplicate Rows based on gmap id as metrics are different
dataframe_MR_RG=dataframe_MR_RG.drop_duplicates(subset=["gmap_id"],keep=False)

In [6]:
#Drop Duplicate Rows based on placekey and keep first row
dataframe_MR_RG=dataframe_MR_RG.drop_duplicates(subset=["placekey"],keep="first")

### Check Unique gmap_ids , placekeys and count of rows

In [7]:
dataframe_MR_RG.shape[0]

996021

In [8]:
len(dataframe_MR_RG["gmap_id"].unique())

996021

In [9]:
len(dataframe_MR_RG["placekey"].unique())

996021

### Fix Null Values

In [10]:
MR_reference_file=pd.read_csv("MR_Reference_File.csv")
MR_reference_file['category']=MR_reference_file['category'].astype(str)
MR_reference_file['category']=MR_reference_file['category'].apply(lambda x:x.strip('][').replace("'","").split(','))
MR_reference_file['category']=MR_reference_file['category'].apply(lambda x:x[0])
MR_reference_file.head()

,gmap_id,category
0,0x88f16e41928ff687:0x883dad4fd048e8f8,Pharmacy
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,Textile exporter
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Korean restaurant
3,0x80c2c89923b27a41:0x32041559418d447,Fabric store
4,0x80c2c632f933b073:0xc31785961fe826a6,Fabric store


In [11]:
RG_reference_file=pd.read_csv("RG_Refernce_File.csv")
RG_reference_file.head()

,placekey,location_name,street_address,city,region,postal_code,latitude,longitude
0,222-222@39p-8k2-mtv,United States Postal Service (USPS),100 6th Ave,Kaltag,AK,99748,64.324568,-158.723724
1,222-222@39s-pzs-wrk,United States Postal Service (USPS),123 Takotna Ave,Mc Grath,AK,99627,62.951525,-155.591123
2,222-222@39x-s8g-yy9,United States Postal Service (USPS),50 Maraq Rd,Napakiak,AK,99634,60.696470,-161.981376
3,222-222@39x-sn7-y35,United States Postal Service (USPS),250 Off Doops St,Akiak,AK,99552,60.910895,-161.223008
4,222-222@39y-p4c-ndv,United States Postal Service (USPS),101 Alstrom St,Saint Marys,AK,99658,62.048097,-163.192118


In [12]:
dataframe_MR_RG=pd.merge(dataframe_MR_RG,MR_reference_file,on="gmap_id")
dataframe_MR_RG.drop(columns=["category_x"],inplace=True)
dataframe_MR_RG.rename(columns={
    "category_y":"category"
},inplace=True)

In [13]:
dataframe_MR_RG=pd.merge(dataframe_MR_RG,RG_reference_file,on="placekey")

In [14]:
dataframe_MR_RG.drop(columns=["location_name_x","street_address_x","city_x","region_x","postal_code_x","latitude_x","longitude_x"],inplace=True)
dataframe_MR_RG.rename(columns={
    "location_name_y":"location_name",
    "street_address_y":"street_address",
    "city_y":"city",
    "region_y":"region",
    "postal_code_y":"postal_code",
    "latitude_y":"latitude",
    "longitude_y":"longitude"
},inplace=True)

### Single Placekey

In [15]:
dataframe_MR_RG

,gmap_id,placekey,Lead,url,Community,Month,spend_by_transaction_intermediary,online_spend,online_transactions,Week Start,...,Average Rating,Count of Reviews,category,location_name,street_address,city,region,postal_code,latitude,longitude
0,0x8636d33deda334eb:0x8539697e533cad66,zzw-222@8f2-d5r-4y9,women-led,https://www.google.com/maps/place//data=!4m2!3...,No Community,"['2019-02-01', ' 2019-04-01', ' 2019-06-01', '...",{'No intermediary'},0,0,"['2019-01-28', '2019-02-04', '2019-02-11', '20...",...,"[5.0, 1.0, 5.0, 2.0, 5.0, 5.0, 5.0, 1.0, 5.0, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",Medical supply store,Accucare Medical,9011 Linwood Ave,Shreveport,LA,71106,32.419479,-93.762377
1,0x88f5a28359cb48cd:0xacc5f8fde9522f87,zzw-223@8g6-ycf-2ff,No Lead,https://www.google.com/maps/place//data=!4m2!3...,No Community,"['2019-03-01', ' 2019-04-01', ' 2019-11-01']",{'No intermediary'},0,0,"['2019-02-25', '2019-03-04', '2019-03-11', '20...",...,"[5.0, 1.0]","[2, 2]",Bank,Gwinnett Community Bank,2775 Buford Hwy,Duluth,GA,30096,34.007993,-84.133536
2,0x883c09638f405513:0x8f0177cc8b7a3593,222-222@63j-rkb-6hq,No Lead,https://www.google.com/maps/place//data=!4m2!3...,No Community,"['2019-01-01', ' 2019-02-01', ' 2019-03-01', '...",{'No intermediary'},0,0,"['2018-12-31', '2019-01-07', '2019-01-14', '20...",...,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",Auto repair shop,Lyric Auto,12953 Kramer Rd,Bowling Green,OH,43402,41.341448,-83.648889
3,0x87faaf9c6de5ad57:0x4ef0ddb309ab3c0d,224-222@5p9-j47-3yv,No Lead,https://www.google.com/maps/place//data=!4m2!3...,No Community,"['2019-01-01', ' 2019-02-01', ' 2019-03-01', '...",{'No intermediary'},0,0,"['2018-12-31', '2019-01-07', '2019-01-14', '20...",...,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]","[2, 2, 2, 2, 2, 2, 2]",Pharmacy,Denver Drug,340 S State St,Denver,IA,50622,42.670611,-92.337478
4,0x87faea62c2d207cf:0x519b41cdfe5036b5,zzw-222@5p9-pcj-kmk,No Lead,https://www.google.com/maps/place//data=!4m2!3...,No Community,"['2020-07-01', ' 2020-08-01', ' 2021-08-01']",{'No intermediary'},0,0,"['2020-06-29', '2020-07-06', '2020-07-13', '20...",...,"[2.0, 5.0, 2.0, 5.0, 5.0, 5.0, 5.0]","[2, 4, 2, 2, 2, 2, 2]",Country club,Meadowbrook Country Club,716 W 6th St,Sumner,IA,50674,42.854989,-92.104478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996016,0x885c24bf5b5bdddb:0x50aad89b7c2bbfee,zzw-228@8g9-m9q-2zf,No Lead,https://www.google.com/maps/place//data=!4m2!3...,No Community,"['2019-01-01', '2019-02-01', '2019-03-01', '20...","[{'No intermediary': 3265.06, 'Postmates': 55....","[55.58, 213.91, 97.36, 76.58, 0.0, 22.71, 0.0,...","[2.0, 7.0, 3.0, 2.0, 0.0, 1.0, 0.0, 0.0, 3.0, ...","['2018-12-31', '2019-01-07', '2019-01-14', '20...",...,"[5.0, 2.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, ...",Fast food restaurant,Chick-fil-A,7600 Kingston Pike Rm NO1320,Knoxville,TN,37919,35.925465,-84.038429
996017,0x8752ed42358df8a7:0x884b43d7e950e6b4,224-222@5qc-wq9-pvz,No Lead,https://www.google.com/maps/place//data=!4m2!3...,No Community,"['2020-05-01', '2020-06-01', '2020-07-01', '20...","[{'No intermediary': 2602.61, 'Visa': 28.18}, ...","[28.18, 27.3, 296.71, 218.88, 186.89, 456.3, 1...","[2.0, 2.0, 9.0, 15.0, 9.0, 16.0, 11.0, 31.0, 1...","['2020-04-27', '2020-05-04', '2020-05-11', '20...",...,"[5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 4.6666666666666...","[1, 4, 1, 1, 1, 1, 3, 1, 2, 2, 1, 1, 1, 1, 2, 1]",Gas station,Chevron,2474 S 7200 W,Magna,UT,84044,40.716347,-112.063224
996018,0x880e31992911c02b:0xd6d8c546635a2507,222-222@5pw-6ct-jy9,No Lead,https://www.google.com/maps/place//data=!4m2!3...,No Community,"['2019-01-01', '2019-02-01', '2019-03-01', '20...","[{'No intermediary': 3750.42, 'Square': 255.17...","[31.44, 0.0, 59.5, 0.0, 0.0, 0.0, 34.46, 81.01...","[2.0, 0.0, 5.0, 0.0, 0.0, 0.0, 2.0, 5.0, 1.0, ...","['2018-12-31', '2019-01-07', '2019-01-14', '20...",...,"[5.0,

In [16]:
for index in dataframe_MR_RG.loc[dataframe_MR_RG["placekey"].str.contains("]")].index:
    Month_list=dataframe_MR_RG.loc[index,"Month"]
    Month_list=Month_list.replace('\\','')
    Month_list=Month_list.split(' "\'", \',\', \' \', "\'", \' \', ')
    Updated_Month_list=[]
    for Month in Month_list:
        Month=re.sub(r'[^a-zA-Z0-9-\s]+', '',Month).replace(" ","")
        Updated_Month_list.append(Month)
    Updated_Month_list_v2=[]
    if(len(Updated_Month_list)==1):
        Updated_Month_list_v2=[Month[i:i+10] for i in range(0, len(Month), 10)]
    else:
        for Month in Updated_Month_list:
            if(len(Month)!=10):
                temp=[Month[i:i+10] for i in range(0, len(Month), 10)]
                for check in temp:
                    Updated_Month_list_v2.append(check)
            else:
                Updated_Month_list_v2.append(Month)
    #break
    dataframe_MR_RG.loc[index,"Month"]=str(Updated_Month_list_v2)

In [17]:
dataframe_MR_RG['placekey']=dataframe_MR_RG['placekey'].apply(lambda x:x.strip('][').replace("'","").split(','))
dataframe_MR_RG['placekey']=dataframe_MR_RG['placekey'].apply(lambda x:x[0])

In [18]:
dataframe_MR_RG=dataframe_MR_RG[[
    "gmap_id","placekey","location_name","street_address","city","region","postal_code","latitude","longitude",
    "Lead",'Community',"category",'url',
    'Month', 'spend_by_transaction_intermediary','online_spend', 'online_transactions', 
    'Week Start','Sum_spent_by_week', 'Average_spent_per_transaction_by_week', 
    'Date','Average Rating', 'Count of Reviews'
]]

In [19]:
dataframe_MR_RG.head(2)

,gmap_id,placekey,location_name,street_address,city,region,postal_code,latitude,longitude,Lead,...,Month,spend_by_transaction_intermediary,online_spend,online_transactions,Week Start,Sum_spent_by_week,Average_spent_per_transaction_by_week,Date,Average Rating,Count of Reviews
0,0x8636d33deda334eb:0x8539697e533cad66,zzw-222@8f2-d5r-4y9,Accucare Medical,9011 Linwood Ave,Shreveport,LA,71106,32.419479,-93.762377,women-led,...,"['2019-02-01', ' 2019-04-01', ' 2019-06-01', '...",{'No intermediary'},0,0,"['2019-01-28', '2019-02-04', '2019-02-11', '20...","[104.3, 34.35, 158.12, 0.0, 70.83, 0.0, 0.0, 0...","[34.77, 4.91, 22.59, 0.0, 17.71, 0.0, 0.0, 0.0...","['2015-11-09', '2016-04-25', '2016-05-16', '20...","[5.0, 1.0, 5.0, 2.0, 5.0, 5.0, 5.0, 1.0, 5.0, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,0x88f5a28359cb48cd:0xacc5f8fde9522f87,zzw-223@8g6-ycf-2ff,Gwinnett Community Bank,2775 Buford Hwy,Duluth,GA,30096,34.007993,-84.133536,No Lead,...,"['2019-03-01', ' 2019-04-01', ' 2019-11-01']",{'No intermediary'},0,0,"['2019-02-25', '2019-03-04', '2019-03-11', '20...","[0.0, 35.0, 25.0, 0.0, 0.0, 325.99, 109.36, 0....","[0.0, 5.0, 3.57, 0.0, 0.0, 46.57, 15.62, 0.0, ...","['2019-07-22', '2020-09-14']","[5.0, 1.0]","[2, 2]"


### Expand Meta_and_Reviews

In [20]:
total_2=dataframe_MR_RG[["gmap_id",'Date','Average Rating', 'Count of Reviews']]

In [21]:
total_2.head()

,gmap_id,Date,Average Rating,Count of Reviews
0,0x8636d33deda334eb:0x8539697e533cad66,"['2015-11-09', '2016-04-25', '2016-05-16', '20...","[5.0, 1.0, 5.0, 2.0, 5.0, 5.0, 5.0, 1.0, 5.0, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,0x88f5a28359cb48cd:0xacc5f8fde9522f87,"['2019-07-22', '2020-09-14']","[5.0, 1.0]","[2, 2]"
2,0x883c09638f405513:0x8f0177cc8b7a3593,"['2012-11-12', '2014-11-10', '2014-12-22', '20...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
3,0x87faaf9c6de5ad57:0x4ef0ddb309ab3c0d,"['2013-07-29', '2016-06-06', '2016-08-22', '20...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]","[2, 2, 2, 2, 2, 2, 2]"
4,0x87faea62c2d207cf:0x519b41cdfe5036b5,"['2013-08-26', '2017-07-10', '2018-04-09', '20...","[2.0, 5.0, 2.0, 5.0, 5.0, 5.0, 5.0]","[2, 4, 2, 2, 2, 2, 2]"


In [22]:
#Date
total_2.loc[:,'Date']=total_2['Date'].apply(lambda x:x.strip('][').replace("'","").split(','))
total_2["Date"]=total_2["Date"].apply(lambda x:[temp.strip() for temp in x])

In [23]:
#Average Rating
total_2.loc[:,'Average Rating']=total_2['Average Rating'].apply(lambda x:x.strip('][').replace("'","").split(','))
total_2.loc[:,'Average Rating']=total_2['Average Rating'].apply(lambda x:[float(temp) for temp in x])
total_2.loc[:,'Average Rating']=total_2['Average Rating'].apply(lambda x:[round(temp,2) for temp in x])

In [24]:
#Count of Reviews
total_2.loc[:,'Count of Reviews']=total_2['Count of Reviews'].apply(lambda x:x.strip('][').replace("'","").split(','))
total_2.loc[:,'Count of Reviews']=total_2['Count of Reviews'].apply(lambda x:[float(temp) for temp in x])

In [25]:
total_2=total_2.explode(["Date","Average Rating","Count of Reviews"])

In [26]:
total_2.rename(columns={
    "Date":"Week_Start_MR",
    "Average Rating":"Average Rating_MR",
    "Count of Reviews":"Count of Reviews_MR"
},inplace=True)

In [27]:
total_2.head(2)

,gmap_id,Week_Start_MR,Average Rating_MR,Count of Reviews_MR
0,0x8636d33deda334eb:0x8539697e533cad66,2015-11-09,5.0,2.0
0,0x8636d33deda334eb:0x8539697e533cad66,2016-04-25,1.0,2.0


### Expand Revenues_and_Geometry

In [28]:
total_3=dataframe_MR_RG[["gmap_id",'Month', 'spend_by_transaction_intermediary','online_spend', 'online_transactions', 
    'Week Start','Sum_spent_by_week', 'Average_spent_per_transaction_by_week']]

In [29]:
#Month
total_3.loc[:,'Month']=total_3['Month'].apply(lambda x:x.strip('][').replace("'","").split(','))
total_3.loc[:,'Month']=total_3['Month'].apply(lambda x:[temp.strip() for temp in x])

In [30]:
#Week_Start
total_3.loc[:,'Week Start']=total_3['Week Start'].apply(lambda x:x.strip('][').replace("'","").split(','))
total_3["Week Start"]=total_3["Week Start"].apply(lambda x:[temp.strip() for temp in x])

In [31]:
#Visits Weekly Level
total_3.loc[:,'Sum_spent_by_week']=total_3['Sum_spent_by_week'].apply(lambda x:x.strip('][').replace("'","").split(','))
total_3.loc[:,'Sum_spent_by_week']=total_3['Sum_spent_by_week'].apply(lambda x:[float(temp) for temp in x])
total_3.loc[:,'Sum_spent_by_week']=total_3['Sum_spent_by_week'].apply(lambda x:[round(temp,2) for temp in x])

In [32]:
#Average Spent By week
total_3.loc[:,'Average_spent_per_transaction_by_week']=total_3['Average_spent_per_transaction_by_week'].apply(lambda x:x.strip('][').replace("'","").split(','))
total_3.loc[:,'Average_spent_per_transaction_by_week']=total_3['Average_spent_per_transaction_by_week'].apply(lambda x:[float(temp) for temp in x])
total_3.loc[:,'Average_spent_per_transaction_by_week']=total_3['Average_spent_per_transaction_by_week'].apply(lambda x:[round(temp,2) for temp in x])

### 1.a Weekly Level

In [33]:
total_3_a=total_3[["gmap_id","Week Start","Sum_spent_by_week","Average_spent_per_transaction_by_week"]]
total_3_a=total_3_a.explode(["Week Start","Sum_spent_by_week","Average_spent_per_transaction_by_week"])
total_3_a.head()

,gmap_id,Week Start,Sum_spent_by_week,Average_spent_per_transaction_by_week
0,0x8636d33deda334eb:0x8539697e533cad66,2019-01-28,104.3,34.77
0,0x8636d33deda334eb:0x8539697e533cad66,2019-02-04,34.35,4.91
0,0x8636d33deda334eb:0x8539697e533cad66,2019-02-11,158.12,22.59
0,0x8636d33deda334eb:0x8539697e533cad66,2019-02-18,0.0,0.0
0,0x8636d33deda334eb:0x8539697e533cad66,2019-02-25,70.83,17.71


In [34]:
total_3_a["Month_Match"]=total_3_a["Week Start"].apply(lambda x:x[0:7]+"-01")
total_3_a

,gmap_id,Week Start,Sum_spent_by_week,Average_spent_per_transaction_by_week,Month_Match
0,0x8636d33deda334eb:0x8539697e533cad66,2019-01-28,104.3,34.77,2019-01-01
0,0x8636d33deda334eb:0x8539697e533cad66,2019-02-04,34.35,4.91,2019-02-01
0,0x8636d33deda334eb:0x8539697e533cad66,2019-02-11,158.12,22.59,2019-02-01
0,0x8636d33deda334eb:0x8539697e533cad66,2019-02-18,0.0,0.0,2019-02-01
0,0x8636d33deda334eb:0x8539697e533cad66,2019-02-25,70.83,17.71,2019-02-01
...,...,...,...,...,...
996020,0x883dc09176756fd9:0x87357cb27eaae39d,2022-08-01,1112.19,9.27,2022-08-01
996020,0x883dc09176756fd9:0x87357cb27eaae39d,2022-08-08,1384.0,9.62,2022-08-01
996020,0x883dc09176756fd9:0x87357cb27eaae39d,2022-08-15,1133.62,13.57,2022-08-01
996020,0x883dc09176756fd9:0x87357cb27eaae39d,2022-08-22,1095.48,8.93,2022-08-01


### 2.b Monthly Level

In [35]:
total_3_b=total_3[["gmap_id",'Month','online_spend', 'online_transactions']]
total_3_b.head()

,gmap_id,Month,online_spend,online_transactions
0,0x8636d33deda334eb:0x8539697e533cad66,"[2019-02-01, 2019-04-01, 2019-06-01, 2019-10-0...",0,0
1,0x88f5a28359cb48cd:0xacc5f8fde9522f87,"[2019-03-01, 2019-04-01, 2019-11-01]",0,0
2,0x883c09638f405513:0x8f0177cc8b7a3593,"[2019-01-01, 2019-02-01, 2019-03-01, 2019-04-0...",0,0
3,0x87faaf9c6de5ad57:0x4ef0ddb309ab3c0d,"[2019-01-01, 2019-02-01, 2019-03-01, 2019-05-0...",0,0
4,0x87faea62c2d207cf:0x519b41cdfe5036b5,"[2020-07-01, 2020-08-01, 2021-08-01]",0,0


In [36]:
total_3_b_a=total_3_b[total_3_b["online_spend"]=="0"].explode(["Month"])
total_3_b_a.head()

,gmap_id,Month,online_spend,online_transactions
0,0x8636d33deda334eb:0x8539697e533cad66,2019-02-01,0,0
0,0x8636d33deda334eb:0x8539697e533cad66,2019-04-01,0,0
0,0x8636d33deda334eb:0x8539697e533cad66,2019-06-01,0,0
0,0x8636d33deda334eb:0x8539697e533cad66,2019-10-01,0,0
0,0x8636d33deda334eb:0x8539697e533cad66,2019-12-01,0,0


In [37]:
total_3_b_b=total_3_b[total_3_b["online_spend"]!="0"]

In [38]:
#Online Spend
total_3_b_b.loc[:,'online_spend']=total_3_b_b['online_spend'].apply(lambda x:x.strip('][').replace("'","").split(','))
total_3_b_b.loc[:,'online_spend']=total_3_b_b['online_spend'].apply(lambda x:[float(temp) for temp in x])
total_3_b_b.loc[:,'online_spend']=total_3_b_b['online_spend'].apply(lambda x:[round(temp,2) for temp in x])

In [39]:
#Online Transactions
total_3_b_b.loc[:,'online_transactions']=total_3_b_b['online_transactions'].apply(lambda x:x.strip('][').replace("'","").split(','))
total_3_b_b.loc[:,'online_transactions']=total_3_b_b['online_transactions'].apply(lambda x:[float(temp) for temp in x])
total_3_b_b.loc[:,'online_transactions']=total_3_b_b['online_transactions'].apply(lambda x:[round(temp,2) for temp in x])

In [40]:
total_3_b_b["Count_1"]=total_3_b_b["online_transactions"].apply(lambda x:len(x))
total_3_b_b["Count_2"]=total_3_b_b["Month"].apply(lambda x:len(x))

In [41]:
total_3_b_b_1=total_3_b_b[total_3_b_b["Count_1"]==total_3_b_b["Count_2"]]
total_3_b_b_1.drop(columns=["Count_1","Count_2"],inplace=True)

In [42]:
total_3_b_b_1=total_3_b_b_1.explode(["Month","online_spend","online_transactions"])

In [43]:
total_3_b_b_1

,gmap_id,Month,online_spend,online_transactions
7,0x80c8e9c312d2a0b9:0xa01533888ebef0d,2019-01-01,0.0,0.0
7,0x80c8e9c312d2a0b9:0xa01533888ebef0d,2019-02-01,0.0,0.0
7,0x80c8e9c312d2a0b9:0xa01533888ebef0d,2019-03-01,0.0,0.0
7,0x80c8e9c312d2a0b9:0xa01533888ebef0d,2019-04-01,0.0,0.0
7,0x80c8e9c312d2a0b9:0xa01533888ebef0d,2019-05-01,0.0,0.0
...,...,...,...,...
996020,0x883dc09176756fd9:0x87357cb27eaae39d,2022-04-01,63.68,2.0
996020,0x883dc09176756fd9:0x87357cb27eaae39d,2022-05-01,10.5,2.0
996020,0x883dc09176756fd9:0x87357cb27eaae39d,2022-06-01,7.77,1.0
996020,0x883dc09176756fd9:0x87357cb27eaae39d,2022-07-01,0.0,0.0


In [44]:
total_3_b_b_2=total_3_b_b[total_3_b_b["Count_1"]!=total_3_b_b["Count_2"]]
total_3_b_b_2

,gmap_id,Month,online_spend,online_transactions,Count_1,Count_2
12981,0x80c2da4e2bb1c0bd:0xd9f1559982453fb7,"[2019-03-01, 2020-01-01, 2019-01-01, 2019-02-0...","[111.13, 50.96]","[3.0, 2.0]",2,7
15554,0x8808a2718aef483b:0xd626b3c37b84f1f7,"[2019-04-01, 2019-05-01, 2019-06-01, 2019-07-0...","[0.0, 44.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",25,30
26534,0x864e7c150bddb88d:0xfde02fba666ba711,"[2019-02-01, 2019-04-01, 2019-05-01, 2019-06-0...","[0.0, 0.0, 0.0, 0.0, 27.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",26,52
59785,0x880a4719012666b1:0xb255742b4262022f,"[2019-04-01, 2019-05-01, 2019-06-01, 2019-07-0...","[23.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",31,35
64802,0x80ea41ed66de9eb1:0x5662a8d1c3ff3c5d,"[2022-07-01, 2022-08-01, 2020-01-01, 2020-02-0...","[0.0, 120.0]","[0.0, 3.0]",2,34
...,...,...,...,...,...,...
948587,0x880654696d27b47b:0x2dfebd28efb44bc3,"[2019-01-01, 2019-02-01, 2019-03-01, 2019-04-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",33,37
953082,0x872b0bc9439f14cb:0xf1789b9024bc6c6,"[2019-01-01, 2019-02-01, 2019-03-01, 2019-04-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",43,48
957246,0x8091ac9f19c997f3:0x532359d4a145fc10,"[2022-05-01, 2022-06-01, 2022-07-01, 2022-08-0...","[35.85, 23.59, 0.0, 0.0]","[2.0, 1.0, 0.0, 0.0]",4,47
966604,0x89c9d6b3e1f8b3a3:0x4a529241d71c421,"[2019-01-01, 2019-02-01, 2019-03-01, 2019-04-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",40,79


In [45]:
for counter in range(0,len(total_3_b_b_2)):
    Month=total_3_b_b_2.iloc[counter]["Month"]
    Count=total_3_b_b_2.iloc[counter]["Count_1"]
    total_3_b_b_2.iloc[counter,1]=str(Month[:Count])

In [46]:
total_3_b_b_2.loc[:,'Month']=total_3_b_b_2['Month'].apply(lambda x:x.strip('][').replace("'","").split(','))

In [47]:
total_3_b_b_2=total_3_b_b_2.explode(["Month","online_spend","online_transactions"])

In [48]:
total_3_b_b=pd.concat([total_3_b_b_1,total_3_b_b_2])
total_3_b_b.drop(columns=["Count_1","Count_2"],inplace=True)

In [49]:
total_3_b=pd.concat([total_3_b_a,total_3_b_b])
total_3_b.head()

,gmap_id,Month,online_spend,online_transactions
0,0x8636d33deda334eb:0x8539697e533cad66,2019-02-01,0,0
0,0x8636d33deda334eb:0x8539697e533cad66,2019-04-01,0,0
0,0x8636d33deda334eb:0x8539697e533cad66,2019-06-01,0,0
0,0x8636d33deda334eb:0x8539697e533cad66,2019-10-01,0,0
0,0x8636d33deda334eb:0x8539697e533cad66,2019-12-01,0,0


### 2.c Spend Transaction Intermediary

In [50]:
total_3_c=total_3[["gmap_id",'Month', 'spend_by_transaction_intermediary']]
total_3_c.head()

,gmap_id,Month,spend_by_transaction_intermediary
0,0x8636d33deda334eb:0x8539697e533cad66,"[2019-02-01, 2019-04-01, 2019-06-01, 2019-10-0...",{'No intermediary'}
1,0x88f5a28359cb48cd:0xacc5f8fde9522f87,"[2019-03-01, 2019-04-01, 2019-11-01]",{'No intermediary'}
2,0x883c09638f405513:0x8f0177cc8b7a3593,"[2019-01-01, 2019-02-01, 2019-03-01, 2019-04-0...",{'No intermediary'}
3,0x87faaf9c6de5ad57:0x4ef0ddb309ab3c0d,"[2019-01-01, 2019-02-01, 2019-03-01, 2019-05-0...",{'No intermediary'}
4,0x87faea62c2d207cf:0x519b41cdfe5036b5,"[2020-07-01, 2020-08-01, 2021-08-01]",{'No intermediary'}


In [51]:
total_3_c_a=total_3_c.loc[~total_3_c["spend_by_transaction_intermediary"].str.contains("]")].explode("Month")
total_3_c_a

,gmap_id,Month,spend_by_transaction_intermediary
0,0x8636d33deda334eb:0x8539697e533cad66,2019-02-01,{'No intermediary'}
0,0x8636d33deda334eb:0x8539697e533cad66,2019-04-01,{'No intermediary'}
0,0x8636d33deda334eb:0x8539697e533cad66,2019-06-01,{'No intermediary'}
0,0x8636d33deda334eb:0x8539697e533cad66,2019-10-01,{'No intermediary'}
0,0x8636d33deda334eb:0x8539697e533cad66,2019-12-01,{'No intermediary'}
...,...,...,...
996011,0x872b00dcb1504521:0x99e177da83a74386,2021-10-01,{'No intermediary'}
996011,0x872b00dcb1504521:0x99e177da83a74386,2022-03-01,{'No intermediary'}
996011,0x872b00dcb1504521:0x99e177da83a74386,2022-06-01,{'No intermediary'}
996015,0x87df2cebab5d7e8f:0xea43e75d285a7aec,2022-07-01,{'No intermediary'}


In [52]:
total_3_c_b=total_3_c.loc[total_3_c["spend_by_transaction_intermediary"].str.contains("]")]
total_3_c_b["spend_by_transaction_intermediary"]=total_3_c_b["spend_by_transaction_intermediary"].apply(lambda x:eval(x))
total_3_c_b=total_3_c_b.explode(["Month","spend_by_transaction_intermediary"])

In [53]:
total_3_c=pd.concat([total_3_c_a,total_3_c_b])
total_3_c.head()

,gmap_id,Month,spend_by_transaction_intermediary
0,0x8636d33deda334eb:0x8539697e533cad66,2019-02-01,{'No intermediary'}
0,0x8636d33deda334eb:0x8539697e533cad66,2019-04-01,{'No intermediary'}
0,0x8636d33deda334eb:0x8539697e533cad66,2019-06-01,{'No intermediary'}
0,0x8636d33deda334eb:0x8539697e533cad66,2019-10-01,{'No intermediary'}
0,0x8636d33deda334eb:0x8539697e533cad66,2019-12-01,{'No intermediary'}


In [54]:
#First Merge
total_3_2=pd.merge(total_3_b,total_3_c,on=["gmap_id","Month"])
total_3_2

,gmap_id,Month,online_spend,online_transactions,spend_by_transaction_intermediary
0,0x8636d33deda334eb:0x8539697e533cad66,2019-02-01,0,0,{'No intermediary'}
1,0x8636d33deda334eb:0x8539697e533cad66,2019-04-01,0,0,{'No intermediary'}
2,0x8636d33deda334eb:0x8539697e533cad66,2019-06-01,0,0,{'No intermediary'}
3,0x8636d33deda334eb:0x8539697e533cad66,2019-10-01,0,0,{'No intermediary'}
4,0x8636d33deda334eb:0x8539697e533cad66,2019-12-01,0,0,{'No intermediary'}
...,...,...,...,...,...
29052261,0x880654696d27b47b:0x2dfebd28efb44bc3,2019-01-01,0.0,0.0,{'No intermediary'}
29052262,0x872b0bc9439f14cb:0xf1789b9024bc6c6,2019-01-01,0.0,0.0,{'No intermediary'}
29052263,0x8091ac9f19c997f3:0x532359d4a145fc10,2022-05-01,35.85,2.0,{'No intermediary'}
29052264,0x89c9d6b3e1f8b3a3:0x4a529241d71c421,2019-01-01,0.0,0.0,{'No intermediary'}


### Total Output

In [55]:
total_3=pd.merge(total_3_a,total_3_2,left_on=["gmap_id","Month_Match"],right_on=["gmap_id","Month"],how="outer")

## Output

In [59]:
total_3.rename(columns={
    "Month_Match":"Month_RG",
    "Week Start":"Week Start_RG"
},inplace=True)

In [62]:
total_3.head()

,gmap_id,Week Start_RG,Sum_spent_by_week,Average_spent_per_transaction_by_week,Month_RG,Month,online_spend,online_transactions,spend_by_transaction_intermediary
0,0x8636d33deda334eb:0x8539697e533cad66,2019-01-28,104.3,34.77,2019-01-01,NaN,NaN,NaN,NaN
1,0x8636d33deda334eb:0x8539697e533cad66,2019-02-04,34.35,4.91,2019-02-01,2019-02-01,0,0,{'No intermediary'}
2,0x8636d33deda334eb:0x8539697e533cad66,2019-02-11,158.12,22.59,2019-02-01,2019-02-01,0,0,{'No intermediary'}
3,0x8636d33deda334eb:0x8539697e533cad66,2019-02-18,0.0,0.0,2019-02-01,2019-02-01,0,0,{'No intermediary'}
4,0x8636d33deda334eb:0x8539697e533cad66,2019-02-25,70.83,17.71,2019-02-01,2019-02-01,0,0,{'No intermediary'}


In [65]:
total_2.head()

,gmap_id,Week_Start_MR,Average Rating_MR,Count of Reviews_MR
0,0x8636d33deda334eb:0x8539697e533cad66,2015-11-09,5.0,2.0
0,0x8636d33deda334eb:0x8539697e533cad66,2016-04-25,1.0,2.0
0,0x8636d33deda334eb:0x8539697e533cad66,2016-05-16,5.0,2.0
0,0x8636d33deda334eb:0x8539697e533cad66,2018-02-12,2.0,2.0
0,0x8636d33deda334eb:0x8539697e533cad66,2018-04-16,5.0,2.0


In [61]:
#Meta and Reviews
total_2["Week_Start_MR"]=total_2["Week_Start_MR"].apply(lambda x:x.strip())

In [63]:
#Revenues and Geometry
total_3["Month_RG"]=total_3["Month_RG"].apply(lambda x:x.strip())

In [64]:
total_3["Week Start_RG"]=total_3["Week Start_RG"].apply(lambda x:x.strip())

In [67]:
total_2.to_csv("Total_2.csv",index=False)

In [68]:
total_3.to_csv("Total_3.csv",index=False)

In [69]:
dataframe_MR_RG[['gmap_id', 'placekey', 'location_name', 'street_address', 'city',
       'region', 'postal_code', 'latitude', 'longitude', 'Lead', 'Community',
       'category', 'url']].to_csv("Total_4.csv",index=False)

In [66]:
total_2_3=pd.merge(total_2,total_3,left_on=["gmap_id","Week_Start_MR"],right_on=["gmap_id","Week Start_RG"],how="outer")

MemoryError: Unable to allocate 4.19 GiB for an array with shape (4, 140675332) and data type object

In [ ]:
total_2_3.loc[~total_2_3["spend_by_transaction_intermediary"].isna(),"spend_by_transaction_intermediary"]=total_2_3.loc[~total_2_3["spend_by_transaction_intermediary"].isna(),"spend_by_transaction_intermediary"].astype(str)

In [ ]:
total_4=dataframe_MR_RG[['gmap_id', 'placekey', 'location_name', 'street_address', 'city',
       'region', 'postal_code', 'latitude', 'longitude', 'Lead', 'Community',
       'category', 'url']]

In [ ]:
total=pd.merge(total_2_3,total_4,on=["gmap_id"])

In [ ]:
total=total[[
    "gmap_id","placekey","location_name","street_address","city","region","postal_code","latitude","longitude",
    "Lead",'Community',"category",'url',
    'Month', 'spend_by_transaction_intermediary','online_spend', 'online_transactions', 
    'Week Start','Sum_spent_by_week', 'Average_spent_per_transaction_by_week', 
    'Date','Average Rating', 'Count of Reviews'
]]

In [ ]:
total.to_csv("MR_RG_Decompressed.csv",index=False)